In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable 
import numpy as np 
import tqdm 
from torch.utils import data 
import librosa
from scipy.io import wavfile
import json
import random

In [ ]:
class Args(object):
    def __init__(self, name="Net", batch_size=8, test_batch_size=50,
                epochs=30, lr=1e-4, optimizer='Adam', momentum=0.9, weight_decay=0.01,
                seed=9, cuda = True):
        self.name = name;
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.weight_decay = weight_decay
        self.optimizer = optimizer
        self.momentum = momentum
        self.seed = seed
        self.cuda = cuda and torch.cuda.is_available()

In [ ]:
class Encoder(nn.Module):
    """Encoder is part of both TrajectoryGenerator and
    TrajectoryDiscriminator"""
    def __init__(
        self, embedding_dim=64, h_dim=64, mlp_dim=1024, num_layers=1,
        dropout=0.0
    ):
        super(Encoder, self).__init__()

        self.mlp_dim = 1024
        self.h_dim = h_dim
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        self.encoder = nn.LSTM(
            embedding_dim, h_dim, num_layers, dropout=dropout
        )

        self.spatial_embedding = nn.Linear(2, embedding_dim)

    def init_hidden(self, batch):
        return (
            torch.zeros(self.num_layers, batch, self.h_dim).cuda(),
            torch.zeros(self.num_layers, batch, self.h_dim).cuda()
        )

    def forward(self, obs_traj):
        """
        Inputs:
        - obs_traj: Tensor of shape (obs_len, batch, 2)
        Output:
        - final_h: Tensor of shape (self.num_layers, batch, self.h_dim)
        """
        # Encode observed Trajectory
        batch = obs_traj.size(1)
        obs_traj_embedding = self.spatial_embedding(obs_traj.reshape(-1, 2))
        obs_traj_embedding = obs_traj_embedding.view(
            -1, batch, self.embedding_dim
        )
        state_tuple = self.init_hidden(batch)
        output, state = self.encoder(obs_traj_embedding, state_tuple)
        final_h = state[0]
        return final_h

In [ ]:
class Decoder(nn.Module):
    """Decoder is part of TrajectoryGenerator"""
    def __init__(
        self, seq_len, embedding_dim=64, h_dim=128, mlp_dim=1024, num_layers=1,
        dropout=0.0, bottleneck_dim=1024,activation='relu', batch_norm=True
    ):
        super(Decoder, self).__init__()

        self.seq_len = seq_len
        self.mlp_dim = mlp_dim
        self.h_dim = h_dim 
        self.embedding_dim = embedding_dim
        self.decoder = nn.LSTM(
            embedding_dim, h_dim, num_layers, dropout=dropout
        )
        
        self.spatial_embedding = nn.Linear(2, embedding_dim)
        self.hidden2pos = nn.Linear(h_dim, 2)
    
    def init_hidden(self, batch):
        return (
            torch.zeros(self.num_layers, batch, self.h_dim).cuda(),
            torch.zeros(self.num_layers, batch, self.h_dim).cuda()
        )
    
    def forward(self, last_pos, state_tuple):
        """
        Inputs:
        - last_pos: Tensor of shape (batch, 2)
        - state_tuple: (hh, ch) each tensor of shape (num_layers, batch, h_dim)
        - pred_traj: tensor of shape (self.seq_len, batch, 2)
        """
        batch = last_pos.size(0)
        pred_traj = []
        decoder_input = self.spatial_embedding(last_pos)
        decoder_input = decoder_input.view(1, batch, self.embedding_dim)

        for _ in range(self.seq_len):
#             print(state_tuple[0].shape)
            output, state_tuple = self.decoder(decoder_input, state_tuple)
            rel_pos = self.hidden2pos(output.view(-1, self.h_dim))
            curr_pos = rel_pos + last_pos

            embedding_input = rel_pos

            decoder_input = self.spatial_embedding(embedding_input)
            decoder_input = decoder_input.view(1, batch, self.embedding_dim)
            pred_traj.append(curr_pos.view(batch, -1))
            last_pos = curr_pos
            
        pred_traj = torch.stack(pred_traj, dim=0)
        return pred_traj, state_tuple[0]


In [ ]:
import json
with open('partition.json') as f:
    partition = json.load(f)

In [ ]:
normalizing_constant = 100
class Dataset(data.Dataset):
    
    def __init__(self, list_IDs, transform=None):
        #'Initialization'
        self.list_IDs = list_IDs
        self.transform = transform
        
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        # Generates one sample of data 
        ID = self.list_IDs[index]
        y = np.load("dataset_archive/trajectory/traj"+str(ID)+".npy")
        y = y[0] - y
        obs_traj = np.load("dataset_archive/obs_traj/obs_traj"+str(ID)+".npy")
        obs_traj = (obs_traj[0] - obs_traj).astype(float)
        obs_traj /= normalizing_constant
        
        if y.shape[0] > 135:
            y = y[0:135,:]
        if y.shape[0] < 135:
            for i in range(0, 135 - y.shape[0]):
                y = np.append(y, y[-1].reshape(1,2), axis=0)
        complete_traj = torch.from_numpy(y).type(torch.FloatTensor)
        complete_traj /= normalizing_constant
        return obs_traj, complete_traj[65:,:]

In [ ]:
args = Args()
params = {'batch_size': args.batch_size,
          'shuffle': True,
          'num_workers': 4}
# Generators
training_set = Dataset(partition['train'])
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['val'])
validation_generator = data.DataLoader(validation_set, **params)

In [ ]:
def make_mlp(dim_list, activation='relu', batch_norm=True, dropout=0):
    layers = []
    for dim_in, dim_out in zip(dim_list[:-1], dim_list[1:]):
        layers.append(nn.Linear(dim_in, dim_out))
        if batch_norm:
            layers.append(nn.BatchNorm1d(dim_out))
        if activation == 'relu':
            layers.append(nn.ReLU())
        elif activation == 'leakyrelu':
            layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(p=dropout))
    return nn.Sequential(*layers)

In [ ]:
class TrajectoryGenerator(nn.Module):
    def __init__(
        self, obs_len, pred_len, embedding_dim=64, encoder_h_dim=64,
        decoder_h_dim=128, mlp_dim=1024, num_layers=2, dropout=0.0, bottleneck_dim=1024,
        activation='relu', batch_norm=False
    ):
        super(TrajectoryGenerator, self).__init__()

        self.obs_len = obs_len
        self.pred_len = pred_len
        self.mlp_dim = mlp_dim
        self.encoder_h_dim = encoder_h_dim
        self.decoder_h_dim = decoder_h_dim
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.bottleneck_dim = 1024

        self.encoder = Encoder(
            embedding_dim=embedding_dim,
            h_dim=encoder_h_dim,
            mlp_dim=mlp_dim,
            num_layers=num_layers,
            dropout=dropout
        )

        self.decoder = Decoder(
            pred_len,
            embedding_dim=embedding_dim,
            h_dim=decoder_h_dim,
            mlp_dim=mlp_dim,
            num_layers=num_layers,
            dropout=dropout,
            bottleneck_dim=bottleneck_dim,
            activation=activation,
            batch_norm=batch_norm
        )
        
        input_dim = encoder_h_dim
        mlp_decoder_context_dims = [
                input_dim, mlp_dim, decoder_h_dim
            ]
        
        self.mlp_decoder_context = make_mlp(
                mlp_decoder_context_dims,
                activation=activation,
                batch_norm=batch_norm,
                dropout=dropout
            )


    def forward(self, obs_traj):
        """
        Inputs:
        - obs_traj: Tensor of shape (obs_len, batch, 2)
        """
        obs_traj = obs_traj.permute(1,0,2)
        batch = obs_traj.size(1)
        # Encode seq
        final_encoder_h = self.encoder(obs_traj)
        
        decoder_h = self.mlp_decoder_context(final_encoder_h.view(-1, self.encoder_h_dim))
        decoder_h = torch.unsqueeze(decoder_h, 0)
        decoder_h = decoder_h.view(self.num_layers, batch, self.decoder_h_dim)
        decoder_c = torch.zeros(
            self.num_layers, batch, self.decoder_h_dim
        ).cuda()

        state_tuple = (decoder_h, decoder_c)
        last_pos = obs_traj[-1]
        # Predict Trajectory

        decoder_out = self.decoder(
            last_pos,
            state_tuple
        )
        pred_traj, final_decoder_h = decoder_out

        return pred_traj

In [ ]:
model = TrajectoryGenerator(65, 70)
if args.cuda:
    model = model.cuda()
train_losses, train_accs = [], []
val_losses, val_accs = [], []
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

In [ ]:
min_test_loss = 10000.
for epoch in range(20):
    # Training
    print("epoch " + str(epoch))
    model.train()
    
    orrect_count, total_loss, total_acc = 0., 0., 0.
    progress_bar = tqdm.tqdm(training_generator, desc='Training')
    
    for batch_idx, (data, target) in enumerate(progress_bar):
        correct_count, total_loss, total_acc = 0., 0., 0.

        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data).float(), Variable(target).float()

        optimizer.zero_grad()
        output = model(data)
        output = output.permute(1,0,2)
        loss_x = F.mse_loss(output[:,:,0], target[:,:,0],reduction="sum") 
        loss_y = F.mse_loss(output[:,:,1], target[:,:,1],reduction="sum")
        loss = loss_x + loss_y
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.data.item())
        
        total_loss += loss.data
        
        progress_bar.clear()
        progress_bar.set_description(
            'Epoch: {} loss: {:.4f}'.format(
                epoch, total_loss / (batch_idx + 1)))
        progress_bar.refresh()
        
    model.eval()
    
    test_loss, correct, acc = 0., 0., 0.
    progress_bar = tqdm.tqdm(validation_generator, desc='Validation')
    with torch.no_grad():
        for data, target in progress_bar:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data).float(), Variable(target).float()

            output = model(data)
            output = output.permute(1,0,2)
            loss_x = F.mse_loss(output[:,:,0], target[:,:,0]) 
            loss_y = F.mse_loss(output[:,:,1], target[:,:,1])
            test_loss += (loss_x + loss_y)
    test_loss /= len(partition['val'])
    val_losses.append(test_loss.item())
    
    if epoch % 10 == 0 and epoch > 0:
        args.lr /= 5
        for param_group in optimizer.param_groups:
            param_group['lr'] = args.lr
            print(param_group['lr'])
    
    progress_bar.clear()
    progress_bar.write(
        '\nEpoch: {} validation test results - Average val_loss: {:.4f}'.format(
            epoch, test_loss))
    
    if (test_loss < min_test_loss):
        torch.save(model, "vision_checkpoint.pkl")
        min_test_loss = test_loss